In [3]:
import numpy as np

# Given Parameters
transmission_rate = [500, 1000, 1500, 2000, 2500, 3000]  # in kb/s
num_devices_list = [50, 100, 150, 200, 250, 300]
task_data = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # in Mbit
max_delay = 0.5  # in seconds
device_computing_capacity = [0.5, 1]  # in GHz
num_of_mecs = 10
mec_computing_capacity = 4  # in GHz
transmission_power = 0.5  # in Watts

# Constants
H_m = 1e-96  # Consumption factor of electricity
alpha_t = 1e-24  # Weighted coefficient of local time cost
alpha_e = 1e-326  # Weighted coefficient of local energy consumption
noise_power_spectral_density = 1e-9  # in Watts/Hz
channel_gain = 2e-10  # 
path_loss_index = 4  # Typical urban area path loss exponent
bandwidth = 1e9  # 1 MHz bandwidth

#fitness function
def fitness_function(x, task_data, transmission_power, bandwidth, noise_power_spectral_density, device_computing_capacity, mec_computing_capacity):
    if x == 0:  # Local computation
        local_time_cost = task_data / device_computing_capacity
        local_energy_cost = H_m * (device_computing_capacity**2) * task_data
        total_cost = alpha_t * local_time_cost + alpha_e * local_energy_cost
    else:  # MEC computation
        transmission_time = task_data / (bandwidth * np.log2(1 + (transmission_power * channel_gain) / (noise_power_spectral_density * bandwidth)))
        mec_time_cost = task_data / mec_computing_capacity
        mec_energy_cost = transmission_power * transmission_time
        total_cost = alpha_t * (transmission_time + mec_time_cost) + alpha_e * mec_energy_cost
    return total_cost

# Adjust QPSO class to use the corrected fitness function
class QPSO:
    def __init__(self, num_devices, max_iter, task_data):
        self.num_devices = num_devices
        self.max_iter = max_iter
        self.task_data = np.array(task_data)  # Convert task data to a numpy array for easy indexing
        self.X = np.random.randint(2, size=(num_devices, max_iter))
        self.P = np.copy(self.X[:, 0:1])
        self.g = None
        self.P_fitness = np.full(num_devices, np.inf)
        self.g_fitness = np.inf

    def run(self):
        for t in range(self.max_iter):
            for i in range(self.num_devices):
                # Adjust to get a single task data size for each device
                task_size = self.task_data[i % len(self.task_data)] * 1e6  # Convert from Mbit to bit
                # Adjust to get a single device computing capacity
                device_capacity = device_computing_capacity[i % len(device_computing_capacity)] * 1e9  # Convert from GHz to Hz
                
                # Evaluate the fitness of the current position Xi
                current_fitness = fitness_function(
                    self.X[i, t], task_size, transmission_power,
                    bandwidth, noise_power_spectral_density, device_capacity, mec_computing_capacity * 1e9  # Convert from GHz to Hz
                )

                # Update personal and global bests based on fitness
                if current_fitness < self.P_fitness[i]:
                    self.P[i, 0] = self.X[i, t]
                    self.P_fitness[i] = current_fitness

                    # Update global best if necessary
                    if current_fitness < self.g_fitness:
                        self.g = np.copy(self.X[:, t])
                        self.g_fitness = current_fitness

            # Update positions for next iteration
            if t < self.max_iter - 1:
                for i in range(self.num_devices):
                    if np.random.random() < 0.5:
                        self.X[i, t+1] = self.g[i]
                    else:
                        self.X[i, t+1] = self.P[i, 0]

        return self.g
    
# Collect the best offloading strategies for each number of devices
best_offloading_strategies = {}

for num_devices in num_devices_list:
    # Initialize QPSO for the current number of devices
    qpso = QPSO(num_devices, max_iter=100, task_data=task_data)
    # Run QPSO and store the best strategy
    best_offloading_strategies[num_devices] = qpso.run()